In [17]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from dotenv import dotenv_values
from pprint import pprint

In [18]:
env_config = dotenv_values("../.env")

In [19]:
es_client = Elasticsearch(f"http://{env_config['ELASTICSEARCH_HOST']}:{env_config['ELASTICSEARCH_PORT']}")

In [20]:
model = SentenceTransformer(env_config["SENTENCE_TRANSFORMER_MODEL"])

## Todos los resultados

In [21]:
params = {
    "index": "boe",
    "query": {"match_all": {}},
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [{'_id': '-qdYwIsBVcU9L_wxWx7Z',
           '_index': 'boe',
           '_score': 1.0,
           '_source': {'doc_id': 'BOE-A-2023-20397',
                       'text': '2.\u2003Don Óscar Raúl Donaire Bravo.'}},
          {'_id': 'AadYwIsBVcU9L_wxXx9d',
           '_index': 'boe',
           '_score': 1.0,
           '_source': {'doc_id': 'BOE-A-2023-20399',
                       'text': 'De acuerdo con lo dispuesto en la Orden '
                               'PRE/189/2016, de\xa017 de febrero, por la que '
                               'se regula el reconocimiento de cualificaciones '
                               'profesionales adquiridas en otros Estados '
                               'miembros de la Unión Europea para el ejercicio '
                               'en España de la profesión de '
                               'Traductor-Intérprete Jurado, se dictó la '
                               'Resolución de\xa02 de marzo de\xa02016, de la '
                  

## Resultados semánticos con KNN

In [22]:
query_text = "cantidad de agua tope"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 10,
    "num_candidates": 10,
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [{'_id': 'I6dYwIsBVcU9L_wxaR-3',
           '_index': 'boe',
           '_score': 0.67210853,
           '_source': {'doc_id': 'BOE-A-2023-20400',
                       'text': 'Número de orden: 1. Puesto de trabajo: Jefe/a '
                               'de Departamento del fondo de cooperación para '
                               'agua y saneamiento (5109142). Dirección de '
                               'Cooperación con América Latina y el Caribe. '
                               'Dirección de la Agencia Española de '
                               'Cooperación Internacional para el Desarrollo. '
                               'Nivel: 30. Complemento Específico: 28.265,44 '
                               'euros. Localidad: Madrid. Subgrupo: A1. '
                               'Adscripción: AE.'}},
          {'_id': 'EqdYwIsBVcU9L_wxZB9S',
           '_index': 'boe',
           '_score': 0.6185689,
           '_source': {'doc_id': 'BOE-A-2023-20399',
                  

## Combinado

In [23]:
query_text = "cantidad de agua"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 10,
    "num_candidates": 10,
    "filter": {
        "match": {
            "text": "liquidez",
        },
    },
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
pprint(res["hits"])

{'hits': [], 'max_score': None, 'total': {'relation': 'eq', 'value': 0}}


## Caso de Uso

In [36]:
query_text = "Convocatiores de oposiciones de bibliotecarios y bibliotecarias de las Cortes"
knn_params = {
    "field": "embedding",
    "query_vector": model.encode(query_text),
    "k": 10,
    "num_candidates": 10,
}
params = {
    "index": "boe",
    "knn": knn_params,
    "_source": ["doc_id", "text"]
}
res = es_client.search(**params)
#pprint(res["hits"])
hits = {}
for hit in res["hits"]["hits"]:
    try:
        hits[hit["_source"]["doc_id"]] = hits[hit["_source"]["doc_id"]] + 1
    except:
        hits[hit["_source"]["doc_id"]] = 1

print("BOE\t -> \tn_apariciones")
for hit in hits:
    print(f"{hit} -> {hits[hit]}")

BOE	 -> 	n_apariciones
BOE-A-2023-20471 -> 2
BOE-A-2023-20470 -> 4
BOE-A-2023-20397 -> 1
BOE-B-2023-28351 -> 1
BOE-B-2023-28350 -> 1
BOE-A-2023-20468 -> 1
